In [ ]:
from collections import deque

class FIFOCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
         # Map: key -> value
        self.cache = {} 
        # tracks insertion order
        self.queue = deque() 

    def get(self, key: int) -> int:
        return self.cache.get(key, -1)

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            # Update value only. Position in queue does NOT change.
            self.cache[key] = value
        else:
            if len(self.cache) >= self.capacity:
                # Evict oldest first in queue
                oldest = self.queue.popleft()
                del self.cache[oldest]
            
            self.cache[key] = value
            self.queue.append(key)

In [ ]:
from collections import OrderedDict

class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = OrderedDict()

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        # Move to end to show it was just used
        self.cache.move_to_end(key)
        return self.cache[key]

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            self.cache.move_to_end(key)
        
        self.cache[key] = value
        
        if len(self.cache) > self.capacity:
            # Pop the first item 
            self.cache.popitem(last=False)

In [ ]:
from collections import defaultdict, OrderedDict

class LFUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.vals = {}
        self.counts = {} 
        self.lists = defaultdict(OrderedDict)
        self.min_freq = 0

    def get(self, key: int) -> int:
        if key not in self.vals:
            return -1
        
        # Update the key's frequency
        self._update_freq(key)
        return self.vals[key]

    def put(self, key: int, value: int) -> None:
        if self.capacity == 0:
            return

        if key in self.vals:
            self.vals[key] = value
            self._update_freq(key)
            return

        if len(self.vals) >= self.capacity:
            # Evict from the list associated with min_freq
            k, _ = self.lists[self.min_freq].popitem(last=False)
            del self.vals[k]
            del self.counts[k]

        # Insert new key
        self.vals[key] = value
        self.counts[key] = 1
        self.lists[1][key] = None
        self.min_freq = 1

    def _update_freq(self, key):
        freq = self.counts[key]
        
        # Remove from current frequency list
        del self.lists[freq][key]
        
        # If that list is empty and it was the minimum, increment min_freq
        if not self.lists[freq] and self.min_freq == freq:
            self.min_freq += 1
        
        # Add to next frequency list
        self.counts[key] += 1
        self.lists[freq + 1][key] = None

In [ ]:
!pip install redis
import redis
import time

class RateLimiter:
    def __init__(self):
     
        self.r = redis.Redis(host='localhost', port=6379, db=0)

    def allow_request(self, user_id: str, limit_per_minute: int) -> bool:
    
        current_minute = int(time.time() // 60)
        
        key = f"rate_limit:{user_id}:{current_minute}"

        current_count = self.r.incr(key)

        if current_count == 1:
            self.r.expire(key, 60)

        # 5. Check limit
        if current_count > limit_per_minute:
            # print(f"User {user_id} has exceeded the rate limit.")
            return False # Block request
        
        return True # Allow request


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import time

class SimpleRedis:
    def __init__(self):
        # key -> (value, expiry_time)
        self.store = {}

    def incr(self, key):
        current_time = time.time()

        # Remove expired key
        if key in self.store:
            value, expiry = self.store[key]
            if expiry and current_time > expiry:
                del self.store[key]

        # Increment
        if key not in self.store:
            self.store[key] = (1, None)
            return 1
        else:
            value, expiry = self.store[key]
            self.store[key] = (value + 1, expiry)
            return value + 1

    def expire(self, key, seconds):
        if key in self.store:
            value, _ = self.store[key]
            self.store[key] = (value, time.time() + seconds)

class SimpleRedis:
    def __init__(self):
        self.store = {}

    def set(self, key, value):
        self.store[key] = value

    def get(self, key):
        return self.store.get(key)

r = SimpleRedis()
r.set("x", 10)
print(r.get("x"))



10
Rate Limiter Results: [True, True, True, True, True, False, False]


In [ ]:


def test_fifo():
    print("\n--- Testing FIFO (Capacity 2) ---")
    cache = FIFOCache(2)
    
    cache.put(1, 1)
    cache.put(2, 2)
    print(f"Added 1, 2.      -> Cache State: {list(cache.cache.keys())}")
    
    cache.get(1) 
    print(f"Accessed 1.      -> Cache State: {list(cache.cache.keys())} (Order shouldn't change)")
    
    cache.put(3, 3) 
    print(f"Added 3.         -> Cache State: {list(cache.cache.keys())}")
    
    # Visual Confirmation
    if 1 not in cache.cache:
        print("VERIFIED: Key 1 is gone.")
    else:
        print("ERROR: Key 1 is still there!")

    assert cache.get(1) == -1
    assert cache.get(2) == 2
    assert cache.get(3) == 3

def test_lru():
    print("\n--- Testing LRU (Capacity 2) ---")
    cache = LRUCache(2)
    
    cache.put(1, 1)
    cache.put(2, 2)
    print(f"Added 1, 2.      -> Cache State: {list(cache.cache.keys())}")
    
    cache.get(1) 
    print(f"Accessed 1.      -> Cache State: {list(cache.cache.keys())} (Note: 1 moved to end/right)")
    
    cache.put(3, 3) 
    print(f"Added 3.         -> Cache State: {list(cache.cache.keys())}")
    
    # Visual Confirmation
    if 2 not in cache.cache:
        print("VERIFIED: Key 2 (Least Recently Used) is gone.")
    else:
        print("ERROR: Key 2 is still there!")

    assert cache.get(2) == -1
    assert cache.get(1) == 1
    assert cache.get(3) == 3

def test_lfu():
    print("\n--- Testing LFU (Capacity 2) ---")
    cache = LFUCache(2)
    
    cache.put(1, 1)
    cache.put(2, 2)
    print(f"Added 1, 2.      -> Cache Keys: {list(cache.vals.keys())} | Frequencies: {cache.counts}")
    
    cache.get(1) 
    print(f"Accessed 1.      -> Cache Keys: {list(cache.vals.keys())} | Frequencies: {cache.counts}")
    
    cache.put(3, 3) 
    print(f"Added 3.         -> Cache Keys: {list(cache.vals.keys())} | Frequencies: {cache.counts}")
    
    # Visual Confirmation
    if 2 not in cache.vals:
        print("VERIFIED: Key 2 (Lowest Freq) is gone.")
    else:
        print("ERROR: Key 2 is still there!")

    assert cache.get(2) == -1
    assert cache.get(1) == 1
    
    print("\n--- Tie-Breaker Test ---")
    cache.get(3)
    print(f"Accessed 3.      -> Cache Keys: {list(cache.vals.keys())} | Frequencies: {cache.counts}")
    
    cache.put(4, 4)
    print(f"Added 4.         -> Cache Keys: {list(cache.vals.keys())} | Frequencies: {cache.counts}")
    
    if 1 not in cache.vals:
        print("VERIFIED: Key 1 (Oldest in tie) is gone.")
    elif 3 not in cache.vals:
         print("VERIFIED: Key 3 (Newest in tie) is gone.")

if __name__ == "__main__":
    test_fifo()
    test_lru()
    test_lfu()
    test_redis_rate_limiter()



--- Testing FIFO (Capacity 2) ---
Added 1, 2.      -> Cache State: [1, 2]
Accessed 1.      -> Cache State: [1, 2] (Order shouldn't change)
Added 3.         -> Cache State: [2, 3]
VERIFIED: Key 1 is gone.

--- Testing LRU (Capacity 2) ---
Added 1, 2.      -> Cache State: [1, 2]
Accessed 1.      -> Cache State: [2, 1] (Note: 1 moved to end/right)
Added 3.         -> Cache State: [1, 3]
VERIFIED: Key 2 (Least Recently Used) is gone.

--- Testing LFU (Capacity 2) ---
Added 1, 2.      -> Cache Keys: [1, 2] | Frequencies: {1: 1, 2: 1}
Accessed 1.      -> Cache Keys: [1, 2] | Frequencies: {1: 2, 2: 1}
Added 3.         -> Cache Keys: [1, 3] | Frequencies: {1: 2, 3: 1}
VERIFIED: Key 2 (Lowest Freq) is gone.

--- Tie-Breaker Test ---
Accessed 3.      -> Cache Keys: [1, 3] | Frequencies: {1: 3, 3: 2}
Added 4.         -> Cache Keys: [1, 4] | Frequencies: {1: 3, 4: 1}
VERIFIED: Key 3 (Newest in tie) is gone.

--- Testing Redis Rate Limiter ---
Request 1: ALLOWED
Request 2: ALLOWED
Request 3: ALLOW